In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

In [3]:
df = pd.read_csv('catalog4/cat4.csv')
df=df.drop(['Unnamed: 0', 'spectrometric_redshift','pred'],axis=1)

In [4]:
df.head()

,galex_objid,sdss_objid,u,g,r,i,z,extinction_u,extinction_g,extinction_r,...,u-g,u-r,u-i,u-z,g-r,g-i,g-z,r-i,r-z,i-z
0,6.374821e+18,1.237667e+18,20.848124,20.591719,20.275167,20.003620,20.300451,0.075471,0.058807,0.040682,...,0.256405,0.572956,0.844503,0.547672,0.316551,0.588099,0.291267,0.271547,-0.025284,-0.296831
1,6.374821e+18,1.237667e+18,20.177017,20.131323,20.197830,20.047293,19.919939,0.082039,0.063924,0.044223,...,0.045694,-0.020813,0.129725,0.257078,-0.066507,0.084030,0.211384,0.150537,0.277891,0.127354
2,6.374821e+18,1.237667e+18,21.058020,20.534431,20.291599,20.443823,20.201283,0.081629,0.063604,0.044001,...,0.523588,0.766420,0.614197,0.856737,0.242832,0.090609,0.333149,-0.152224,0.090317,0.242540
3,6.374821e+18,1.237667e+18,20.866688,20.533670,20.257778,19.890251,19.691072,0.083731,0.065242,0.045134,...,0.333017,0.608910,0.976437,1.175615,0.275892,0.643419,0.842598,0.367527,0.566706,0.199179
4,6.374821e+18,1.237667e+18,20.024755,20.000088,20.171259,20.368889,20.472017,0.083741,0.065250,0.045140,...,0.024668,-0.146503,-0.344133,-0.447262,-0.171171,-0.368801,-0.471930,-0.197630,-0.300758,-0.103128


In [5]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [6]:
train_df, test_df = train_test_split(df, test_size=20)

In [7]:
#train_df.head()
test_df.head()

,galex_objid,sdss_objid,u,g,r,i,z,extinction_u,extinction_g,extinction_r,...,u-g,u-r,u-i,u-z,g-r,g-i,g-z,r-i,r-z,i-z
30733,6.378621e+18,1.237667e+18,20.463758,20.254568,20.215668,20.065311,19.910500,0.068747,0.053567,0.037057,...,0.209190,0.248091,0.398447,0.553259,0.038900,0.189257,0.344069,0.150356,0.305168,0.154812
20812,6.378973e+18,1.237650e+18,20.698353,20.601778,20.230881,20.390915,20.009155,0.169407,0.132001,0.091318,...,0.096575,0.467472,0.307438,0.689198,0.370897,0.210863,0.592623,-0.160034,0.221725,0.381760
17915,6.376756e+18,1.237680e+18,21.790569,21.115210,20.403118,19.725279,19.302748,0.344413,0.268364,0.185653,...,0.675360,1.387451,2.065290,2.487822,0.712091,1.389931,1.812462,0.677839,1.100370,0.422531
12056,3.053408e+18,1.237679e+18,22.007545,22.096613,21.912487,21.123716,20.926476,0.260755,0.203178,0.140558,...,-0.089067,0.095058,0.883829,1.081070,0.184126,0.972897,1.170137,0.788771,0.986012,0.197241
19726,6.387980e+18,1.237680e+18,20.509596,20.164829,19.852970,19.783581,19.785446,0.243176,0.189481,0.131082,...,0.344767,0.656626,0.726015,0.724150,0.311859,0.381248,0.379383,0.069389,0.067524,-0.001865


In [8]:

def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification


def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above



In [9]:
import pandas as pd
df=pd.read_csv('project/dataset/catalog1/cat1.csv')
#df.head()
#df.columns
df=df.drop(['Unnamed: 0', 'spectrometric_redshift','pred'],axis=1)
#df.head()
#df.head()
target=df['class']
target
df=df.drop(['class'],axis=1)


# def check_purity(data):
    
#     label_column = data[:, -1]
#     unique_classes = np.unique(label_column)

#     if len(unique_classes) == 1:
#         return True
#     else:
#         return False
    
# check_purity(df)

In [10]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [11]:

def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
#     data=df
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [12]:

def calculate_accuracy(df, tree):

    df["pred_class"] = df.drop(["class"],axis=1).apply(classify_example, axis=1, args=(tree,))
    
    df["pred_class_correct"] = df["pred_class"] == df["class"]
    print(df.head())
    accuracy = df["pred_class_correct"].mean()
    
    return accuracy

In [119]:
def calculate_metrics(df,tree):
        df["pred_class"] = df.drop(["class"],axis=1).apply(classify_example, axis=1, args=(tree,))
        TP=TN=FP=FN=0
        for i,j in list(zip(df["pred_class"], df["class"])):
            if i==j:
                if i==0:
                    TP+=1
                else:
                    TN+=1
            else:
                if i==0 and j==1:
                    FN+=1
                else:
                    FP+=1
          
        accuracy=(TP+TN)/(TP+TN+FP+FN)
        precision=TP/(TP+FP)
        recall=TP/(TP+FN)
        f1_score=(2*precision*recall)/(precision+recall)
        confusion_matrix=[[TP,FN],[FP,TN]]
        print("Confusion matrix")
        for i in confusion_matrix:
            print(i[0],"\t",i[1])
        print("\naccuracy: ",accuracy,"\n","precison: ",precision,"\n","recall: ",recall,"\n","f1_score: ",f1_score)
                

In [129]:
def upsample(df):
    c0 = 0
    c1 = 0
    for i in df['class']:
        if(i==0):
            c0+=1
        else:
            c1+=1


    print(c0,c1)

    resample_size = c1

    # Shuffle the Dataset.
    shuffled_df = df.sample(frac=1,random_state=4)

    quaser = shuffled_df.loc[shuffled_df['class'] == 1]
    star = shuffled_df.loc[shuffled_df['class'] == 0].sample(n=resample_size,random_state=42, replace=True)

    # Concatenate both dataframes again
    normalized_df = pd.concat([star, quaser])

    return normalized_df



In [131]:
df = pd.read_csv("catalog2/cat2.csv")
#df=df.sample(n=8000)
df=df.drop(['Unnamed: 0', 'spectrometric_redshift','pred'],axis=1)
df_old=df

target=df['class']
target


#df=upsample(df)
#df.head()
#df.head()
target=df['class']
target

train_df, test_df = train_test_split(df, test_size=0.2)

#train_df=df.drop(['class'],axis=1)

Y_train=train_df["class"]
train_df=train_df.drop(["class"],axis=1)
train_df["class"]=Y_train
train_df=upsample(train_df)

#df.shape
#train_df.shape
#train_df.head()

344 2573


In [132]:
tree = decision_tree_algorithm(train_df)
tree
#accuracy = calculate_accuracy(test_df, tree)

{'g-i <= -0.09377574920659804': [{'nuv-z <= 2.7542996406555007': [{'z <= 20.28216361999515': [{'r-z <= -0.1253986358642507': [0.0,
        {'r-i <= -0.06039524078370029': [1.0, 0.0]}]},
      {'r-z <= 0.7547922134399023': [{'nuv-r <= 2.2298316955565998': [0.0,
          1.0]},
        1.0]}]},
    1.0]},
  {'u-g <= 0.7262191772461026': [{'r-z <= 0.907662391662548': [{'r <= 21.291113853454604': [1.0,
        {'nuv-z <= 1.1186943054198508': [0.0, 1.0]}]},
      {'sdss_objid <= 1.2376625013477217e+18': [{'r-i <= 0.45965671539304864': [1.0,
          0.0]},
        {'r-z <= 1.2627820968628498': [1.0, 0.0]}]}]},
    {'r-z <= 1.3352718353271449': [{'fuv-r <= -0.08090782165524857': [{'u-g <= 0.920425415039098': [1.0,
          0.0]},
        {'fuv-g <= 0.2887878417969496': [0.0, 1.0]}]},
      {'z <= 18.993367195129352': [0.0,
        {'fuv-u <= 0.5051317214965518': [1.0, 0.0]}]}]}]}]}

In [133]:
accuracy = calculate_accuracy(test_df, tree)

      Unnamed: 0.1          galex_objid           sdss_objid          u  \
3167         54347  3115895144343146722  1237661070869856551  20.973497   
947          16796  2914570167250855275  1237678661430346295  22.222012   
2336         34524  6375982373790550449  1237680245203403061  20.418371   
18             251  6374821280921683069  1237667254011756581  20.570784   
1748         27976  3054568783792312295  1237679476396785958  21.782209   

              g          r          i          z  extinction_u  extinction_g  \
3167  20.620829  20.194695  19.769392  19.528002      0.132505      0.103247   
947   22.492468  22.024233  21.904295  23.080427      0.092869      0.072363   
2336  19.992119  19.851780  19.788740  19.511915      0.146797      0.114383   
18    20.146570  20.118689  19.813540  19.594950      0.095954      0.074767   
1748  21.286785  21.553823  21.521538  21.180134      0.214411      0.167068   

             ...               r-z       i-z   fuv-nuv     fuv-u    

In [134]:
accuracy

0.8916323731138546

In [135]:
calculate_metrics(test_df,tree)

Confusion matrix
60 	 66
13 	 590

accuracy:  0.8916323731138546 
 precison:  0.821917808219178 
 recall:  0.47619047619047616 
 f1_score:  0.6030150753768845


In [136]:
from sklearn.metrics import accuracy_score,auc
print(accuracy_score(test_df["pred_class"],test_df["class"]))
#print(auc(test_df["pred_class"],test_df["class"]))

0.8916323731138546
